In [1]:
import dataIO_funcs
# from datetime import datetime
import datetime
import numpy as np
import dateutil.parser
import os
from snowexsql.data import PointData # Import our class for the points table


# import requests 
# from datetime import datetime
# from bs4 import BeautifulSoup

In [2]:
# specify paths
bucket_name = 'eis-dh-hydro/SNOWEX-HACKWEEK'
LIS_path = f's3://{bucket_name}/2022/ZARR/SURFACEMODEL/LIS_HIST_default_chunks.d01.zarr/'
# LIS_path = f's3://{bucket_name}/2022/ZARR/SURFACEMODEL/LIS_HIST_rechunkedV4.d01.zarr'
SWESARR_url = 'https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease/'

# SWESARR data website
source_repo = 'https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease/'

# specify filters
time_sel='2020-02-08'
lat_range = [38.8, 39.2]
lon_range = [-108.3, -107.5]
model_variable = ['SM_SWE_inst','SM_SnowDepth_inst']
dx = 0.0011
dy = 0.0009
time_buffer_dy = 2


# FOR PULLING SWESARR
#### TAKES A LOOK AT ALL OF THE DATA AVAILABLE, THEN ATTAEMPTS TO PULL THE DATA THAT IS 'TIME_BUFFER_DY' FROM THE SPECIFIED DAY 'TIME_SEL'

In [7]:
# pull the dates that SWESARR occurred
SWESARR_names,SWESARR_dates = dataIO_funcs.get_url_paths(SWESARR_url)

In [8]:
def date_between_prime(start_date, end_date, folder_date,folder_names):
    result = [folder_names[i] for i in range(len(folder_date)) if (folder_date[i] >= start_date and folder_date[i] <= end_date)]
    return result
time_sel_newForm = dateutil.parser.parse(time_sel)
start_date = time_sel_newForm-datetime.timedelta(days=time_buffer_dy)
end_date = time_sel_newForm+datetime.timedelta(days=time_buffer_dy)

final_files = date_between_prime(start_date, end_date, SWESARR_dates, SWESARR_names)
for flight_line in final_files:
    print(flight_line)
    ! wget -r -np -nH --reject "indexd.html*" -e robots=off -r --no-parent -A "*tif" source_repo + flight_line
    
data_files = []
os.chdir(os.getcwd() + flight_line)
for file in glob.glob("*.tif"):
    data_files.append(file)
    
data_files = [data for data in data_files if data[-8:] != '_dem.tif']

/GRMCT2_13802_20006_012_200210_225_XX_01/
--2022-07-14 00:33:23--  http://source_repo/
Resolving source_repo (source_repo)... failed: Name or service not known.
wget: unable to resolve host address ‘source_repo’
--2022-07-14 00:33:23--  http://+/
Resolving + (+)... failed: Name or service not known.
wget: unable to resolve host address ‘+’
--2022-07-14 00:33:23--  http://flight_line/
Resolving flight_line (flight_line)... failed: Name or service not known.
wget: unable to resolve host address ‘flight_line’
/GRMCT2_31601_20006_011_200210_225_XX_01/
--2022-07-14 00:33:24--  http://source_repo/
Resolving source_repo (source_repo)... failed: Name or service not known.
wget: unable to resolve host address ‘source_repo’
--2022-07-14 00:33:24--  http://+/
Resolving + (+)... failed: Name or service not known.
wget: unable to resolve host address ‘+’
--2022-07-14 00:33:24--  http://flight_line/
Resolving flight_line (flight_line)... failed: Name or service not known.
wget: unable to resolve hos

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/model-eval/contributors/jupflug/GRMST1_27501_20006_003_200210_225_XX_01/'

# LIS MODEL READ-IN
#### READS IN THE DATA BASED ON THE LAT/LON/DATE CONSTRAINTS, PROVIDED THE MODEL VARIABLES OF FOCUS
#### WE ALSO GIVE IT THE APPROXIMATE MODEL DX AND DY WE WANT TO MOVE TO WHEN CONVERTING TO A RECTANGULAR GRID

In [3]:
ds_LIS = dataIO_funcs.access_LIS(lon_range,lat_range,time_sel,
                             model_variable,LIS_path,dx,dy,'bilinear')

In [3]:
ds_LIS

NameError: name 'ds_LIS' is not defined

# LIDAR OBSERVATION READ-IN

# POINT AND LAYER DATA READ-IN

In [3]:
from snowexsql.db import get_db
from snowexsql.data import LayerData,PointData# Import the function to get connect to the db
from snowexsql.conversions import query_to_geopandas # Import a useful function to format that data into a dataframe 

db_name = 'snow:hackweek@db.snowexdata.org/snowex' # This is what you will use for all of hackweek to access the db
type_name = 'layer'
var_name = 'swe'

In [6]:
engine, session = get_db(db_name)
q = session.query(LayerData).filter(LayerData.type == 'swe')
# q = q.filter(LayerData.latitude < lat_range[1])
# q = q.filter(LayerData.latitude > lat_range[0])
# q = q.filter(LayerData.longitude < lon_range[1])
# q = q.filter(LayerData.longitude > lon_range[0])

df = query_to_geopandas(q,engine)
session.close()

df

,site_name,date,time_created,time_updated,id,doi,date_accessed,instrument,type,units,...,depth,site_id,pit_id,bottom_depth,comments,sample_a,sample_b,sample_c,value,flags


In [26]:
q



In [15]:
q = session.query(LayerData).filter(LayerData.type == 'density')
df = query_to_geopandas(q, engine)

# Convert density to float
df['value'] = df['value'].astype(float)

# Calculate SWE
swe_lambda = lambda row: row['value'] * (row['depth'] - row['bottom_depth']) / 100
df['swe'] = df.apply(swe_lambda, axis=1)

In [18]:
q = session.query(LayerData)
q.filter(LayerData.latitude < 39.2)

In [3]:
from snowexsql.db import get_db # Import the function to get connect to the db
from snowexsql.data import LayerData # Import our LayerData
from snowexsql.conversions import query_to_geopandas # Import a useful function to format that data into a dataframe 



In [4]:
db_name = 'snow:hackweek@db.snowexdata.org/snowex' # This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name)

In [ ]:
session.query(

In [ ]:
q = session.query(LayerData).filter(LayerData.type == 'density')
df = query_to_geopandas(q, engine)

# Convert density to float
df['value'] = df['value'].astype(float)

# Calculate SWE
swe_lambda = lambda row: row['value'] * (row['depth'] - row['bottom_depth']) / 100
df['swe'] = df.apply(swe_lambda, axis=1)

In [2]:
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData

In [ ]:
db_name = 'snow:hackweek@db.snowexdata.org/snowex'


In [ ]:
url = 'https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease/'
folder_names = get_url_paths(url)
folder_names = folder_names[5:-1]
folder_names = [name[62:] for name in folder_names]
folder_date = [datetime.strptime(date[24:30] , '%y%m%d') for date in folder_names]
# print(folder_date)

In [ ]:
engine, session = get_db(db_name)
session.close()

In [ ]:
source_repo = 'https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease/'
flight_line = '/GRMCT2_31801_20007_016_200211_225_XX_01/'